In [56]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk

In [13]:
#data = requests.get('http://localhost:8080/chat/5/sentiment').json()
#data

In [15]:
import mongoAtlasConnection as mon
db_sent, coll_sent = mon.connectCollection('chat_conversations', 'sentiments')

In [22]:
query=list(coll_sent.find())
#query

In [65]:
users=set([q["idUser"] for q in query])
#query

In [66]:
user_message={}
for u in users:
    user_message[f'{u}']=""

In [67]:
for q in query:
    user_message[str(q["idUser"])]+=" "+q["Message"]

In [68]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
i=0
for m in user_message.values():
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(m)
    clean_text=[ct for ct in tokens if ct not in stop_words]
    user_message[str(i)]=" ".join(clean_text)
    i+=1

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/blancalluch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(user_message.values())
sparse_matrix

<8x177 sparse matrix of type '<class 'numpy.int64'>'
	with 210 stored elements in Compressed Sparse Row format>

In [71]:
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=user_message.keys())

In [72]:
df.head()

,6am,aa,alcoholics,alien,and,angry,anonymous,appreciated,ass,awesome,...,we,went,what,whats,work,would,www,you,youre,youve
0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0
2,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,1,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,1


In [73]:
from sklearn.metrics.pairwise import cosine_similarity as distance

# Compute Cosine Similarity matrix (or selected distance)
similarity_matrix = distance(df, df)
print(similarity_matrix)

[[1.         0.09847319 0.06356417 0.13655775 0.         0.
  0.         0.09697623]
 [0.09847319 1.         0.10327956 0.05547002 0.0246183  0.02020305
  0.         0.10504515]
 [0.06356417 0.10327956 1.         0.10741723 0.06356417 0.02608203
  0.         0.        ]
 [0.13655775 0.05547002 0.10741723 1.         0.06827887 0.11206636
  0.         0.        ]
 [0.         0.0246183  0.06356417 0.06827887 1.         0.14920942
  0.         0.03232541]
 [0.         0.02020305 0.02608203 0.11206636 0.14920942 1.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  1.         0.13130643]
 [0.09697623 0.10504515 0.         0.         0.03232541 0.
  0.13130643 1.        ]]


In [75]:
sim_df = pd.DataFrame(similarity_matrix, columns=user_message.keys(), index=user_message.keys())
display(sim_df)

,0,1,2,3,4,5,6,7
0,1.000000,0.098473,0.063564,0.136558,0.000000,0.000000,0.000000,0.096976
1,0.098473,1.000000,0.103280,0.055470,0.024618,0.020203,0.000000,0.105045
2,0.063564,0.103280,1.000000,0.107417,0.063564,0.026082,0.000000,0.000000
3,0.136558,0.055470,0.107417,1.000000,0.068279,0.112066,0.000000,0.000000
4,0.000000,0.024618,0.063564,0.068279,1.000000,0.149209,0.000000,0.032325
5,0.000000,0.020203,0.026082,0.112066,0.149209,1.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.131306
7,0.096976,0.105045,0.000000,0.000000,0.032325,0.000000,0.131306,1.000000


In [95]:
import numpy as np
np.fill_diagonal(sim_df.values, 0) # Remove diagonal max values and set those to 0
list(zip(list(sim_df['0'].sort_values(ascending=False)[:3].index),list(sim_df['0'].sort_values(ascending=False)[:3])))



[('3', 0.1365577483997838),
 ('1', 0.09847319278346618),
 ('7', 0.0969762275752854)]